# ImportExcel Module

Started by Doug Finke with significant input from James O'Neill, ImportExcel enables automation of Excel workbooks

## Installation
Lets start by installing it (we will also check that you have dbatools too!)

In [ ]:
$Modules = 'ImportExcel','dbatools'
foreach($Module in $Modules){
    if(Get-Module $Module -ListAvailable -ErrorAction SilentlyContinue){
        Write-Output "We will update the module $Module"
        Update-Module $Module
    } else {
        Write-Output "We will install the module $Module"
        Install-Module $Module -Scope CurrentUser
    }
}


## Listing the commands

The module has a significant number of commands

In [ ]:
Get-Command -Module ImportExcel

## Start with something easy

Let's start of by showing the power of PowerShell and the ImportExcel module

First lets get the logins on an instance with dbatools

In [ ]:
Get-DbaLogin -SqlInstance localhost | Format-Table

Of course, your user wants them in Excel!!

With ImportExcel Module, this is easy

In [ ]:
Get-DbaLogin -SqlInstance localhost | Export-Excel -Path C:\temp\xlsx\Logins.xlsx
Invoke-Item C:\temp\xlsx\Logins.xlsx

Thats cool, <u>but we want more</u>.

It would be good to auto format the columns so that they fit

In [ ]:
Get-DbaLogin -SqlInstance localhost | Export-Excel -Path C:\temp\xlsx\Logins.xlsx -AutoSize
Invoke-Item C:\temp\xlsx\Logins.xlsx

 Thats neat but I want to be able to filter by the columns

In [ ]:
Get-DbaLogin -SqlInstance localhost | Export-Excel -Path C:\temp\xlsx\Logins.xlsx -AutoSize -AutoFilter
Invoke-Item C:\temp\xlsx\Logins.xlsx

I want a title and to be able to name the worksheet

and I want it to just open with one line

In [ ]:
Get-DbaLogin -SqlInstance localhost | Export-Excel -Path C:\temp\xlsx\Logins.xlsx -AutoSize -AutoFilter -Title "The Logins" -WorksheetName "Logins" -Show 

Hang on a minute, what happened there?

It created a new worksheet and added it onto the existing workbook but it made it second

Lets fix that with the `MoveToStart` parameter, We'll make the background yellow so its obvious

In [ ]:
Get-DbaLogin -SqlInstance localhost | Export-Excel -Path C:\temp\xlsx\Logins.xlsx -AutoSize -AutoFilter -Title "The Logins" -WorksheetName "Logins" -MoveToStart -TitleBackgroundColor yellow -FreezeTopRow -Show 

## Use cases

This is really neat, its a simple way of getting the results of any PowerShell command, quickly and easily into an Excel sheet for sorting

### Services

In [ ]:
Get-Service | Export-Excel -Path C:\temp\xlsx\Services.xlsx -AutoSize -AutoFilter -WorksheetName "Services" -MoveToStart -FreezeTopRow -Show 

### Processes

This takes a little minute

In [ ]:
Get-Process | Export-Excel -Path C:\temp\xlsx\Processes.xlsx -AutoSize -AutoFilter -WorksheetName "Processes" -MoveToStart -FreezeTopRow -Show 

### Event Logs

In [ ]:
Get-WinEvent -LogName System -MaxEvents 100  | Export-Excel -Path C:\temp\xlsx\EventLogs.xlsx -AutoSize -AutoFilter -WorksheetName "System" -MoveToStart -FreezeTopRow 
Get-WinEvent -LogName Application -MaxEvents 100  | Export-Excel -Path C:\temp\xlsx\EventLogs.xlsx -AutoSize -AutoFilter -WorksheetName "Application" -MoveToStart -FreezeTopRow -Show

###  SQL Error Logs

In [ ]:
Get-DbaErrorLog -SqlInstance localhost  | Export-Excel -Path C:\temp\xlsx\SQLErrorLogs.xlsx -AutoSize -AutoFilter -WorksheetName "SQL Error Logs" -MoveToStart -FreezeTopRow -Show


## How About doing it all in one ?

In [ ]:
Get-Service | Export-Excel -Path C:\temp\xlsx\Incident.xlsx -AutoSize -AutoFilter -WorksheetName "Services" -MoveToStart -FreezeTopRow
#Get-Process | Export-Excel -Path C:\temp\xlsx\Incident.xlsx -AutoSize -AutoFilter -WorksheetName "Processes" -MoveToStart -FreezeTopRow
'just a dummy entry' | Export-Excel -Path C:\temp\xlsx\Incident.xlsx -AutoSize -AutoFilter -WorksheetName "Processes" -MoveToStart -FreezeTopRow
Get-WinEvent -LogName System -MaxEvents 100  | Export-Excel -Path C:\temp\xlsx\Incident.xlsx -AutoSize -AutoFilter -WorksheetName "System" -MoveToStart -FreezeTopRow
Get-WinEvent -LogName Application -MaxEvents 100  | Export-Excel -Path C:\temp\xlsx\Incident.xlsx -AutoSize -AutoFilter -WorksheetName "Application" -MoveToStart -FreezeTopRow
Get-DbaErrorLog -SqlInstance localhost  | Export-Excel -Path C:\temp\xlsx\Incident.xlsx -AutoSize -AutoFilter -WorksheetName "SQL Error Logs" -MoveToStart -FreezeTopRow -Show


## More Information needed

So thats useful but how about if we add a front page which gives some more information?

In [ ]:
$SqlInstance = 'Beard-Desktop'
$Date = Get-Date -Format yyyyMMdd
$Directory = 'C:\temp\xlsx\'
$ExcelFile = $Directory + $Date + '_Incident_For_' + $SqlInstance + '.xlsx'

Get-Service | Export-Excel -Path $ExcelFile -AutoSize -AutoFilter -WorksheetName "Services" -MoveToStart -FreezeTopRow
#Get-Process | Export-Excel -Path $ExcelFile -AutoSize -AutoFilter -WorksheetName "Processes" -MoveToStart -FreezeTopRow
'just a dummy entry' | Export-Excel -Path $ExcelFile -AutoSize -AutoFilter -WorksheetName "Processes" -MoveToStart -FreezeTopRow
Get-WinEvent -LogName System -MaxEvents 100  | Export-Excel -Path $ExcelFile -AutoSize -AutoFilter -WorksheetName "System" -MoveToStart -FreezeTopRow
Get-WinEvent -LogName Application -MaxEvents 100  | Export-Excel -Path $ExcelFile -AutoSize -AutoFilter -WorksheetName "Application" -MoveToStart -FreezeTopRow
Get-DbaErrorLog -SqlInstance localhost  | Export-Excel -Path $ExcelFile -AutoSize -AutoFilter -WorksheetName "SQL Error Logs" -MoveToStart -FreezeTopRow 

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the System Event Log, Application Event Log, Services,Processes and SQL Error Log for $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

Close-ExcelPackage $excel

Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile

## Charts and Pivot Tables

So we have made a good start but we know that users will want there to be pictures as well as data.

Lets get some data from the Bicycle Manufacturer

In [ ]:
$Query = "SELECT Top 25000 PROD.Name, 
OrdHead.OrderDate ,
Details.OrderQty
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID"
$results = Invoke-DbaQuery -SqlInstance localhost -Database AdventureWorks2017 -Query $Query 
$results | Export-Excel -Path C:\temp\xlsx\query.xlsx -Show

Thats a bit annoying but it is due to the datatable object that is returned. We can fix that using `Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors`

In [ ]:
$Query = "SELECT Top 25000 PROD.Name, 
OrdHead.OrderDate ,
Details.OrderQty
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID"
$results = Invoke-DbaQuery -SqlInstance localhost -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors
$results  | Export-Excel -Path C:\temp\xlsx\query.xlsx -Show

Lets format it nicely again

In [ ]:
$results | Export-Excel -Path C:\temp\xlsx\formattedquery.xlsx -AutoSize -AutoFilter -WorksheetName 'A Query' -TableName 'BeardsAreAwesome' -TableStyle Dark1 -Show

## Chart

We can create a chart with our data like this

In [ ]:
$Query = "SELECT  
YEAR(OrdHead.OrderDate) AS Year,
SUM(Details.OrderQty) AS TotalSales
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID
GROUP BY YEAR(OrdHead.OrderDate)
ORDER BY YEAR(OrdHead.OrderDate) 
"
$results = Invoke-DbaQuery -SqlInstance localhost -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors
$excel = $results | Export-Excel -Path C:\temp\xlsx\chartedquery.xlsx -AutoSize -AutoFilter -TableName 'BeardsAreAwesome' -TableStyle Dark1 -AutoNameRange  -PassThru 
$chart = New-ExcelChartDefinition -Title BeardyChart -ChartType ColumnClustered -XRange Year -YRange TotalSales  -Column 4 -NoLegend
Export-Excel -ExcelPackage $excel -ExcelChartDefinition $chart  -Show

I'm a DBA, I like doughnuts :-)

In [ ]:
$Query = "SELECT  
YEAR(OrdHead.OrderDate) AS Year,
SUM(Details.OrderQty) AS TotalSales
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID
GROUP BY YEAR(OrdHead.OrderDate)
ORDER BY YEAR(OrdHead.OrderDate) 
"
$results = Invoke-DbaQuery -SqlInstance localhost -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors
$excel = $results | Export-Excel -Path C:\temp\xlsx\chartedquery.xlsx -AutoSize -AutoFilter -TableName 'BeardsAreAwesome' -TableStyle Dark1 -AutoNameRange  -PassThru 
$chart = New-ExcelChartDefinition -Title 'Beardy Chart of Total Sales By Year' -ChartType Doughnut -XRange Year -YRange TotalSales  -Column 4  -ShowPercent
Export-Excel -ExcelPackage $excel -ExcelChartDefinition $chart  -Show

## Pivot

We can also create Pivot Tables

In [ ]:
$Query = "SELECT PROD.Name, 
OrdHead.OrderDate ,
Details.OrderQty
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID"
$results = Invoke-DbaQuery -SqlInstance localhost -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors
$excel = $results | Export-Excel -Path C:\temp\xlsx\pivotedquery.xlsx -AutoSize -AutoFilter -WorksheetName 'BaseQuery' -TableName 'BeardsAreAwesome' -TableStyle Dark1 -PassThru
$PivotTable = New-PivotTableDefinition -PivotTableName BeardyPivot -SourceWorkSheet BaseQuery -PivotData @{"OrderQty" = "Sum"} -PivotRows OrderDate ,Name 

Export-Excel -ExcelPackage $excel -PivotTableDefinition $PivotTable -Show

It would be nice if we could add the Magic date to the Pivot as well

In [ ]:
$excel = $results | Export-Excel -Path C:\temp\xlsx\pivoteddatedquery.xlsx -AutoSize -AutoFilter -WorksheetName 'BaseQuery' -TableName 'BeardsAreAwesome' -TableStyle Dark1 -PassThru
$PivotTable = New-PivotTableDefinition -PivotTableName BeardyPivot -SourceWorkSheet BaseQuery -PivotData @{"OrderQty" = "Sum"} -PivotRows OrderDate ,Name -GroupDateRow OrderDate -GroupDatePart Years, Months, Days -PivotTableStyle Medium1 

Export-Excel -ExcelPackage $excel -PivotTableDefinition $PivotTable -Show

and how about a Pivot Chart also just to make it beautiful

In [ ]:
$excel = $results | Export-Excel -Path C:\temp\xlsx\pivoteddatedchartedquery.xlsx -AutoSize -AutoFilter -WorksheetName 'BaseQuery' -TableName 'BeardsAreAwesome' -TableStyle Dark1 -PassThru
$PivotTable = New-PivotTableDefinition -PivotTableName BeardyPivot -SourceWorkSheet BaseQuery -PivotData @{"OrderQty" = "Sum"} -PivotRows OrderDate ,Name -GroupDateRow OrderDate -GroupDatePart Years, Months, Days -PivotTableStyle Medium1 -IncludePivotChart -ChartTitle BeardyChart -ChartType ColumnClustered 

Export-Excel -ExcelPackage $excel -PivotTableDefinition $PivotTable -Show

## Formatting

Excel enables you to do custom formatting. ImportExcel does the same.

Lets check permissions on an instance and colour code the results depending on the role membership

In [ ]:
$ExcelDirectory = 'c:\temp\xlsx\' # Alter this to the directory you want the file created
$SQlinstance = 'localhost'  # Alter this for the SQL Instance you want to get permissions for

Write-Output "Processing $sqlinstance"

$InstanceName = $SQlinstance.Split('\').Split('.').Split('\').Split(',') -join '_'
$ExcelFile = $ExcelDirectory + '\' + $InstanceName + '_Permssions_OneTab_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$WorkSheetName = "Permissions"

    $excel = Get-DbaUserPermission -SqlInstance $sqlinstance | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter -PassThru
  
    $rulesparam = @{
        Address   = $excel.Workbook.Worksheets[$WorkSheetName].Dimension.Address
        WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 
        RuleType  = 'Expression'      
    }

    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("sysadmin",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("db_owner",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER LOGINS",$E1)))' -BackgroundColor PaleGreen 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER SECURABLES",$E1)))' -BackgroundColor PowderBlue 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB ROLE MEMBERS",$E1)))' -BackgroundColor GoldenRod 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB SECURABLES",$E1)))' -BackgroundColor BurlyWood 

    Close-ExcelPackage $excel


Write-Output ""
Write-Output "Finished Processing $sqlinstance"

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the User Permissions for each database on $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

$TitleSheet.Cells["B3"].Value = "The Cells are colour coded as follows :-"
Set-ExcelRange -Worksheet $TitleSheet -Range "B3" -Bold -FontSize 18 
$TitleSheet.Cells["E5"].Value = "The Yellow Cells show members of the sysadmin role who have permission to do and access anything on the instance "
$TitleSheet.Cells["E6"].Value = "The Green Cells show the logins on the server"
$TitleSheet.Cells["E7"].Value = "The Blue Cells show the instance level permissions that have been granted to the logins"
$TitleSheet.Cells["E8"].Value = "The Orange Cells show the database role membership for the login"
$TitleSheet.Cells["E9"].Value = "The Brown Cells show specific database permissions that have been granted for the logins"

$TitleSheet.Cells["B11"].Value = "You can filter by Database on the Object column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C11" -FontSize 18

$TitleSheet.Cells["B12"].Value = "You can filter by User/Group/Login on the Member column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C12" -FontSize 18

Set-ExcelRange -Worksheet $TitleSheet -Range  "C5" -BackgroundColor Yellow
Set-ExcelRange -Worksheet $TitleSheet -Range  "C6" -BackgroundColor PaleGreen
Set-ExcelRange -Worksheet $TitleSheet -Range  "C7" -BackgroundColor PowderBlue 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C8" -BackgroundColor GoldenRod 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C9" -BackgroundColor BurlyWood 

Close-ExcelPackage $excel


Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile